In [51]:
import numpy as np
import pandas as pd

import os
import glob

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor

from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')

In [53]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/Xtest.csv', index_col=0)

In [3]:
def write_submission(pred_test, filename):
    sample = pd.read_csv('data/sample_submission.csv', index_col='Id')
    sample['Times'] = pred_test
    sample.to_csv(filename)

# Статистика паролей

In [4]:
numbers = '0123456789'
Aletters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
aletters = 'abcdefghijklmnopqrstuvwxyz'
vowel = 'AaEeIiOoUuYy'
#consonant = 'bcdfghjklmnpqrstvwxz'
characters = '!*-._@\')$?#&%=;^\\+~['



def transformAa(data):
    X = pd.DataFrame()
    X['Password']=data['Password'].fillna('NaN')
    X['password']=X['Password'].apply(lambda x: str(x).lower())
    X['len']=X['Password'].apply(lambda x: len(x))
    

    for i in numbers:
        X[i]=X['Password'].apply(lambda x: x.count(i))
    for i in Aletters:
        X[i]=X['Password'].apply(lambda x: x.count(i))
    for i in aletters:
        X[i]=X['Password'].apply(lambda x: x.count(i))
    for i in characters:
        X[i]=X['Password'].apply(lambda x: x.count(i))
        
    X['max']=X[list(numbers + Aletters + aletters + characters)].max(axis=1)
    
    X['numbers'] = X[list(numbers)].sum(axis=1)/X['len']
    X['Aletters'] = X[list(Aletters)].sum(axis=1)
    X['aletters'] = X[list(aletters)].sum(axis=1)
    X['vowel'] = X[list(vowel)].sum(axis=1) / (X['Aletters'] + X['aletters']+1)
    
    X['Aletters1'] = X[list(Aletters)].sum(axis=1)/X['len']
    X['Aletters2'] = X[list(Aletters)].sum(axis=1)/(X['Aletters'] + X['aletters']+1)
    
    X['characters'] = X[list(characters)].sum(axis=1)/X['len']
    
        
    #return X.drop('Password', axis=1)
    return X[['Password', 'max', 'len', 'numbers', 'Aletters', 'Aletters1','Aletters2', 'aletters', 'vowel', 'characters']]

In [5]:
XX = transformAa(train)
y = np.log1p(train['Times'])

In [11]:
test_f = transformAa(test)

# Внешние данные

In [28]:
for i in glob.glob('data/exte/*.txt'):
    file = pd.read_csv(i, header=None, sep='\t')
    passwords = set(file[0].values)
    train[os.path.split(i)[1].split('.')[0]+'wout']=train['Password'].apply(lambda x: x in passwords)
    test[os.path.split(i)[1].split('.')[0]+'wout']=test['Password'].apply(lambda x: x in passwords)
    
    print(np.nan in passwords)
    print(os.path.split(i)[1].split('.')[0],file.shape, len(passwords))
    
for i in glob.glob('data/exte/*.txt'):
    file = pd.read_csv(i, header=None, sep='\t', dtype=str)
    passwords = set(file.fillna('NaN')[0].values)
    train[os.path.split(i)[1].split('.')[0]]=train['Password'].fillna('NaN').apply(lambda x: str(x).lower() in passwords)
    test[os.path.split(i)[1].split('.')[0]]=test['Password'].fillna('NaN').apply(lambda x: str(x).lower() in passwords)
    print(np.nan in passwords)
    print(os.path.split(i)[1].split('.')[0],file.shape, len(passwords))
    
    
for i in glob.glob('data/exte/*.txt'):
    file = pd.read_csv(i, header=None, sep='\t').fillna('NaN')
    passwords = dict(zip(file[0], file.index))
    train[os.path.split(i)[1].split('.')[0]+'wout1']=train['Password'].map(passwords).fillna(0).astype('int')
    test[os.path.split(i)[1].split('.')[0]+'wout1']=test['Password'].map(passwords).fillna(0).astype('int')
    
    print(os.path.split(i)[1].split('.')[0],file.shape, len(passwords))
    del passwords
    
    
for i in glob.glob('data/exte/*.txt'):
    file = pd.read_csv(i, header=None, sep='\t', dtype=str).fillna('NaN')
    passwords = dict(zip(file[0], file.index))
    train[os.path.split(i)[1].split('.')[0]+'1']=train['Password'].map(lambda x: str(x).lower()).map(passwords).fillna(0).astype('int')
    test[os.path.split(i)[1].split('.')[0]+'1']=test['Password'].map(lambda x: str(x).lower()).map(passwords).fillna(0).astype('int')

    print(os.path.split(i)[1].split('.')[0],file.shape, len(passwords))
    del passwords

False
10-million-password-list-top-1000 (1000, 1) 1000
False
10-million-password-list-top-10000 (10000, 1) 10000
True
10-million-password-list-top-100000 (100000, 1) 100000
True
10-million-password-list-top-1000000 (999999, 1) 999998
False
10-million-password-list-top-500 (500, 1) 500
True
10k-most-common (10000, 1) 10000
False
best1050 (1049, 1) 1049
False
common-passwords-win (815, 1) 815
True
words (466544, 1) 466542
False
10-million-password-list-top-1000 (1000, 1) 1000
False
10-million-password-list-top-10000 (10000, 1) 10000
False
10-million-password-list-top-100000 (100000, 1) 100000
False
10-million-password-list-top-1000000 (999999, 1) 999998
False
10-million-password-list-top-500 (500, 1) 500
False
10k-most-common (10000, 1) 10000
False
best1050 (1049, 1) 1049
False
common-passwords-win (815, 1) 815
False
words (466544, 1) 466542
10-million-password-list-top-1000 (1000, 1) 1000
10-million-password-list-top-10000 (10000, 1) 10000
10-million-password-list-top-100000 (100000, 1)

In [29]:
rockyou_dict = {}

with open('data/rockyou-withcount.txt', 'r', encoding='utf-8', errors='ignore') as file:
    for row in file:
        times_pass = row.split()
        if len(times_pass)==2:
            times, password = times_pass
            if password not in rockyou_dict:
                rockyou_dict[password]=times

In [30]:
train['rockyou'] = train['Password'].map(rockyou_dict).fillna(0).astype('int')
test['rockyou'] = test['Password'].map(rockyou_dict).fillna(0).astype('int')

In [34]:
cols = ['10-million-password-list-top-10000wout1',
        '10-million-password-list-top-100000wout1',
        '10-million-password-list-top-1000000wout1',
        '10-million-password-list-top-500wout1', 'wordswout1',
        '10-million-password-list-top-10001',
        '10-million-password-list-top-1000001',
        '10-million-password-list-top-10000001',
        '10-million-password-list-top-5001', '10k-most-common1', 'best10501',
        'common-passwords-win1', 'words1', 'rockyou',
        '10-million-password-list-top-1000wout',
        '10-million-password-list-top-100000wout',
        '10-million-password-list-top-1000000wout',
        '10-million-password-list-top-500']

def transform(data):
    X=data.fillna('NaN').copy()
    return X[cols]

In [35]:
train.head()

,Password,Times,10-million-password-list-top-1000wout,10-million-password-list-top-10000wout,10-million-password-list-top-100000wout,10-million-password-list-top-1000000wout,10-million-password-list-top-500wout,10k-most-commonwout,best1050wout,common-passwords-winwout,...,10-million-password-list-top-10001,10-million-password-list-top-100001,10-million-password-list-top-1000001,10-million-password-list-top-10000001,10-million-password-list-top-5001,10k-most-common1,best10501,common-passwords-win1,words1,rockyou
0,631XniVx2lS5I,2,False,False,False,True,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0
1,LEGIT747,1,False,False,False,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0
2,742364es,1,False,False,False,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0
3,3846696477,1,False,False,False,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0
4,laurahop,2,False,False,False,True,False,False,False,False,...,0,0,0,508057,0,0,0,0,0,0


In [36]:
my_train_exte_rych = transform(train)
my_test_exte_rych = transform(test)

In [38]:
XX_fext = pd.concat([XX, my_train_exte_rych], axis=1).drop(columns='Password').astype('float64')
test_fext = pd.concat([test_f, my_test_exte_rych], axis=1).drop(columns='Password').astype('float64')

# Tfidf

In [43]:
countv1 = TfidfVectorizer(analyzer='char',lowercase=False,ngram_range=(1,1), max_features=100)
countv2 = TfidfVectorizer(analyzer='char',lowercase=False,ngram_range=(2,2), max_features=1000)
countv3 = TfidfVectorizer(analyzer='char',lowercase=False,ngram_range=(3,3), max_features=1000)
countv4 = TfidfVectorizer(analyzer='char',lowercase=False,ngram_range=(4,4), max_features=1000)
countv5 = TfidfVectorizer(analyzer='char',lowercase=False,ngram_range=(5,5), max_features=1000)

In [44]:
XX1 = countv1.fit_transform(XX['Password'])
XX2 = countv2.fit_transform(XX['Password'])
XX3 = countv3.fit_transform(XX['Password'])
XX4 = countv4.fit_transform(XX['Password'])
XX5 = countv5.fit_transform(XX['Password'])

XX_f = hstack([XX1, XX2, XX3, XX4, XX5, XX_fext])

In [45]:
test1 = countv1.transform(test_f['Password'])
test2 = countv2.transform(test_f['Password'])
test3 = countv3.transform(test_f['Password'])
test4 = countv4.transform(test_f['Password'])
test5 = countv5.transform(test_f['Password'])

test_ff = hstack([test1, test2, test3, test4, test5, test_fext])

# Модель

In [40]:
lgb = LGBMRegressor(n_estimators=200, max_depth=20, n_jobs=-1)
lgb.fit(XX_fext, y)
pred_test = lgb.predict(test_fext)
pred_test = [i if i>np.log1p(1) else np.log1p(1) for i in pred_test]
pred_test = np.expm1(pred_test)
write_submission(pred_test, 'submissions/extdata_ry_stat1.csv') # private = 0.00268, public = 0.00403

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.1, max_depth=20,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=200, n_jobs=-1, num_leaves=31, objective=None,
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [46]:
lgb = LGBMRegressor(n_estimators=200, max_depth=20, n_jobs=-1)
lgb.fit(XX_f, y)
pred_test = lgb.predict(test_ff)
pred_test = [i if i>np.log1p(1) else np.log1p(1) for i in pred_test]
pred_test = np.expm1(pred_test)
write_submission(pred_test, 'submissions/extdata_ry_stat_tfidf.csv') # private = 0.00262, public = 0.00423

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.1, max_depth=20,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=200, n_jobs=-1, num_leaves=31, objective=None,
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [52]:
%%time
rf = RandomForestRegressor(n_estimators=10, max_depth=10, n_jobs=-1)
rf.fit(XX_fext, y)
pred_test = rf.predict(test_fext)
pred_test = [i if i>np.log1p(1) else np.log1p(1) for i in pred_test]
pred_test = np.expm1(pred_test)
write_submission(pred_test, 'submissions/extdata_ry_stat_rf.csv') #private=0.00134,public=0.00306

CPU times: user 3min 2s, sys: 1.54 s, total: 3min 4s
Wall time: 59.2 s
